# Introduction
This notebook shows how to connect to Alpaca and receive live price data from it. We then run a simple strategy using this live data feed for a fixed period of time and then show some results.  

In [ ]:
%use roboquant
Welcome()

By default the required Alpaca feed classes are not imported, so we need to do that first.

In [ ]:
import org.roboquant.feeds.alpaca.*

# Setup
We now create, as usual, the strategy we want to test. There is nothing specific to Alapca Feed that the strategy needs to be aware of. 
And we create an instance of Roboquant with the strategy we just defined and some metrics to later see how we have done. Again this is all independent of the data feed we are going to use.

Please note we create a very fast EMA (Exponential Moving Average) Crossover and that already after 3 observations might start generating signals. Not a very common practice, but for the purpose of this demo we don't want to have to wait too long before any results can be observed.  

In [ ]:
val strategy = EMACrossover(3, 5, minEvents = 3)
val roboquant = Roboquant(strategy, AccountSummary(), ProgressMetric())

# Feed
Now we create an instance of the AlpacaFeed and subscribe to one or more assets. We need to provide the API Key and Secret to authenticate against the Alpaca API. We can do this by:

* Setting the environment or JVM variables APCA_API_KEY_ID and APCA_API_SECRET_KEY
* Or pass them as parameters to the *AlpacaFeed* constructor. If you take this approach, make sure you don't accidentally share your notebook with anyone.
* Or set them in the dotenv file found in the same directory

Also if you are going to run this notebook yourself, make sure you **use KEY/SECRET for paper trading since roboquant is still very much in Beta**.


In [ ]:
// val feed = AlpacaFeed("your api key", "you secret")

val feed = AlpacaFeed()

Alpaca offers the prices for many stocks. For this notebook we're going to limit that a bit, we just get all the stocks who's symbol name starts with "A" and then subscribe to them. Alpaca has a generous API rate limit, so it is possible for example to subscribe to all S&P 500 assets.

In [ ]:
val assets = feed.assets.filter { it.symbol.startsWith("A") }
feed.subscribe(assets)
assets.summary().print(20)

# Run
We have all the components assembled that we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. 
We'll run it for 30 minutes, but you can change this. But remember the EMA Crossover is only going to evaluate after at least three observations.

If the time that is displayed in the progress bar looks off by a few hours, please realize that roboquant uses a timezone independent representation for all internal time processing (same as UTC).

In [ ]:
val timeFrame = TimeFrame.nextMinutes(30)
roboquant.run(feed, timeFrame)
feed.disconnect()

# Results
The run has completed, lets see a few result. Of course if the run was outside trading hours without any price action, there will not be much to see.

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val logger = roboquant.logger
logger.summary().print()

In [ ]:
logger.getMetricNames().summary()

# Charts
Also lets plot two charts. See also the visualization notebook for examples of how to use charts to display results

In [ ]:
val accountValue = logger.getMetric("account.value")
MetricChart(accountValue, useTime = false)

In [ ]:
TradeChart(account.trades)